In [1]:
# Format file 'Crime_Statistics_-_Incidents_and_rates_by_offence_category.csv'
import pandas as pd
import numpy as np

crimeStats = pd.read_csv('data/Crime_Statistics_-_Incidents_and_rates_by_offence_category.csv')

# Extract first geography string
geo = crimeStats.GEOGRAPHY.str.split(',').values
crimeStats['GEOGRAPHY2'] = [row[0] for row in geo]

# Format Year
from datetime import datetime
crimeStats['year'] = [datetime.strptime(y, '%m/%d/%Y %I:%M:%S %p').year for y in crimeStats.YEAR]
crimeStats.set_index(['GEOGRAPHY', 'year'], inplace=True)

rates = crimeStats.pivot_table(values='RATES', index=crimeStats.index, columns='VIOLATIONS', dropna=False)
ratesTransp = rates.values.transpose()
rateKeys = rates.keys()
newCrimeStats = crimeStats.pivot_table(values='INCIDENTS', index=crimeStats.index, columns='VIOLATIONS', dropna=False)

for i, key in enumerate(rateKeys):
    newCrimeStats['Rate ' + key] = ratesTransp[i]

idx = newCrimeStats.index.values
idx = np.array([np.array(row) for row in idx])
idx = idx.transpose()

newCrimeStats['geography'] = idx[0]
newCrimeStats['year'] = idx[1]

In [2]:
counties = pd.read_csv('data/proc/ns_counties.csv')
newCrimeStats['CO_CODE'] = ['' for row in newCrimeStats.values]
geo = newCrimeStats.geography.str.split(',').values
line = -1

def getCounty(name):
    if name == 'Amherst':
        return 'CU'
    elif name == 'Barrington':
        return 'SH'
    elif name == 'Bridgewater':
        return 'LU'
    elif name == 'Eskasoni':
        return 'CB'
    elif name == 'Indian Brook':
        return 'HN'
    elif name == 'Kentville':
        return 'KI'
    elif name == 'Meteghan (Clare)':
        return 'DI'
    elif name == 'Millbrook':
        return 'CO'
    elif name == 'New Glasgow Region':
        return 'PI'
    elif name == 'Oxford County':
        return 'CU'
    elif name == 'Parrsboro':
        return 'CU'
    elif name == 'Port Hawkesbury':
        return 'CB'
    elif name == 'Springhill':
        return 'CU'
    elif name == 'Stellarton':
        return 'PI'
    elif name == 'Trenton':
        return 'PI'
    elif name == 'Truro':
        return 'CO'
    elif name == 'Westville':
        return 'PI'
    elif name == 'Windsor':
        return 'HN'
    return ''

for i, row in newCrimeStats.iterrows():
    line += 1
    for j, county in counties.iterrows():
        name = county['CO_DESC'].lower()
        code = county['CO_CODE']

        if name in geo[line][0].lower():
            newCrimeStats.loc[i, 'CO_CODE'] = code

line = -1

for i, row in newCrimeStats.iterrows():
    line += 1
    if row['CO_CODE'] != '':
        continue
    newCrimeStats.loc[i, 'CO_CODE'] = getCounty(geo[line][0])

newCrimeStats.drop(newCrimeStats[newCrimeStats['CO_CODE'] == ''].index, inplace=True)
# newCrimeStats.loc[newCrimeStats['CO_CODE'] == '',]

In [3]:
newCrimeStats = newCrimeStats.fillna(0)

newCrimeStats.drop(['geography'], axis=1, inplace=True)
newCrimeStats = newCrimeStats.groupby(['CO_CODE', 'year']).agg('sum')

# Normalization
from sklearn import preprocessing

for key in newCrimeStats.keys():
    newCrimeStats[key] = newCrimeStats[key] / newCrimeStats[key].max()

idx = newCrimeStats.index.values
idx = np.array([np.array(row) for row in idx])
idx = idx.transpose()

newCrimeStats['CO_CODE'] = idx[0]
newCrimeStats['CO_DESC'] = [counties.loc[counties['CO_CODE'] == code, 'CO_DESC'].values[0] for code in idx[0]]
newCrimeStats['YEAR'] = idx[1]

newCrimeStats.to_csv('data/proc/crime_stats_by_offence_category.csv', index=False)

VIOLATIONS    Total other Criminal Code violations  \
CO_CODE year                                         
AP      2010                              0.045443   
        2011                              0.050927   
        2012                              0.058240   
        2013                              0.057717   
        2014                              0.050666   
        2015                              0.044920   
        2016                              0.044920   
        2017                              0.038652   
AT      2010                              0.050144   
        2011                              0.054845   
        2012                              0.072081   
        2013                              0.057717   
        2014                              0.067119   
        2015                              0.040742   
        2016                              0.048054   
        2017                              0.045443   
CB      2010                